<a href="https://colab.research.google.com/github/QiaoLin22/MASTER-LLM-DL/blob/main/PDF_Query_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.3 MB/s eta 0:00:00


In [3]:
from PyPDF2 import PdfReader
!pip install langchain-community
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.6 MB/s eta 0:00:00


In [6]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [7]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [8]:
# location of the pdf file/files.
reader = PdfReader('/content/gdrive/My Drive/data/CMMI.pdf')

In [9]:
reader

In [10]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [11]:
raw_text[:100]

' \n \n \n \n \nCMMI® for Development, Version 1.3 \nCMMI-DEV, V1.3  \nCMMI Product Team \n \nImproving proces'

In [12]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits.

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [14]:
len(texts)
texts[0]

'CMMI® for Development, Version 1.3 \nCMMI-DEV, V1.3  \nCMMI Product Team \n \nImproving processes for developing better products and services \nNovember 2010 \nTECHNICAL REPORT \nCMU/SEI-2010-TR -033 \nESC-TR-2010- 033 \n \nSoftware Engineering Process Management Program \nUnlimited distribution subject to the copyright. \n \nhttp://www.sei.cmu.edu This report was prepared for the \nSEI Administrative Agent  \nESC/XPK\n5 Eglin Street  \nHanscom AFB, MA 01731-2100 \nThe ideas  and findings in this report should not be construed as an official DoD position. It is \npublished in the interest of scientific and technical information exchange. \nThis work is sponsored by the U.S. Department of Defense. The Software Engineering Institute is a \nfederally funded research and development center sponsored by the U.S. Department of Defense. \nCopyright 2010 Carnegie Mellon University. NO WARRANTY  \nTHIS CARNEGIE MELLON UNIVERSITY AND SOFTWARE ENGINEERING INSTITUTE'

In [15]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

<ipython-input-15-f838166cf7bd>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [16]:
docsearch = FAISS.from_texts(texts, embeddings)

In [17]:
docsearch

In [18]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [19]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

<ipython-input-19-25e042d5cc81>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  chain = load_qa_chain(OpenAI(), chain_type="stuff")
<ipython-input-19-25e042d5cc81>:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: 

In [20]:
query = "who are the authors of the article?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

<ipython-input-20-e33c64a809b8>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=docs, question=query)


' Kirsten Dauplaise, Mike Evanoo, Rich Frost, Brian Gallagher, Sally Godfrey, Stephen Gristock, Nils Jacobsen, Steve Kapurch, Chris Moore, Wendell Mullison, David (Mike) Phillips, Robert Rassa, Karen Richter, Warren Schwoemeyer, John Scibilia, Dave Swidorsky, Mary Van Tyne, Rawdon Young, So Norimatsu, Seven Ou, Ricardo Panero Lamothe, Mary Lynn Russo, Winfried Russwurm, Tomas San Feliu, Dan Bennett, Will Hayes, Rick Hefner, Jim Kubeck, Alice Parry, Mary Lynn Penn, Kathy Smith, Craig Hollenbach, Bradley Nelson, Lawrence Osiecki, Timothy Salerno, Sandy Shrum, Nidhi Srivastava, Elizabeth Sumpter, David Swidorsky, Rhonda Brown, Mary Beth Chrissis, Eileen Forrester, Mike Konrad.'

In [21]:
query = "What is CMMI?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' CMMI stands for Capability Maturity Model Integration and it is a collection of best practices that help organizations improve their processes. It is developed by product teams with members from industry, government, and the Software Engineering Institute (SEI). CMMI provides guidance for applying best practices in different areas, such as development and service delivery, to help organizations improve their overall performance and quality.'

In [23]:
query = "What is this article about?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' This article is about the CMMI for Development model, which is a framework used for process improvement in development environments. It also provides information on the history and evolution of process improvement models.'